노트북이 너무 쌓이면 새로운 노트북으로 갈아타줍니다.

# 변수 모으기 2

# IMPORT & LOAD DATA

In [2]:
import load_dtypes as ld
import warnings
warnings.filterwarnings(action='ignore')

import os, sys
import time
import datetime as dt

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN

from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, roc_auc_score, roc_curve, precision_recall_curve, recall_score, precision_score

import lightgbm as lgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import StratifiedKFold

In [3]:
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------

In [4]:
TRAIN_P_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/train_problem_data.csv'
TRAIN_Q_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/train_quality_data.csv'
TRAIN_E_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/train_err_data.csv'
TEST_Q_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/test_quality_data.csv'
TEST_E_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/test_err_data.csv'
SUBMISSION_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/sample_submission.csv'

In [146]:
%%time
train_p = ld.load_dtypes(TRAIN_P_PATH)
train_q = ld.load_dtypes(TRAIN_Q_PATH)
train_e = ld.load_dtypes(TRAIN_E_PATH)
test_q = ld.load_dtypes(TEST_Q_PATH)
test_e = ld.load_dtypes(TEST_E_PATH)
submission = pd.read_csv(SUBMISSION_PATH)

C:\Users\Wyatt\wyatt37/Data/systemError/train_problem_data.csv
C:\Users\Wyatt\wyatt37/Data/systemError/train_quality_data.csv
C:\Users\Wyatt\wyatt37/Data/systemError/train_err_data.csv
C:\Users\Wyatt\wyatt37/Data/systemError/test_quality_data.csv
C:\Users\Wyatt\wyatt37/Data/systemError/test_err_data.csv
Wall time: 1min 20s


In [147]:
train_p.shape, train_q.shape, train_e.shape, test_q.shape, test_e.shape, submission.shape

((5429, 2),
 (828624, 16),
 (16554663, 6),
 (747972, 16),
 (16532648, 6),
 (14999, 2))

In [148]:
train_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5429 entries, 0 to 5428
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  5429 non-null   int16
 1   time     5429 non-null   int64
dtypes: int16(1), int64(1)
memory usage: 53.1 KB


In [149]:
train_q.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828624 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   time        828624 non-null  int64   
 1   user_id     828624 non-null  int16   
 2   fwver       788544 non-null  category
 3   quality_0   684192 non-null  float32 
 4   quality_1   828624 non-null  int16   
 5   quality_2   788511 non-null  float32 
 6   quality_3   828624 non-null  int8    
 7   quality_4   828624 non-null  int8    
 8   quality_5   828604 non-null  category
 9   quality_6   828624 non-null  int16   
 10  quality_7   828624 non-null  category
 11  quality_8   828624 non-null  category
 12  quality_9   828624 non-null  category
 13  quality_10  828624 non-null  category
 14  quality_11  828624 non-null  int8    
 15  quality_12  828624 non-null  int8    
dtypes: category(6), float32(2), int16(3), int64(1), int8(4)
memory usage: 28.9 MB


In [150]:
train_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16554663 entries, 0 to 16554662
Data columns (total 6 columns):
 #   Column    Dtype   
---  ------    -----   
 0   user_id   int16   
 1   time      int64   
 2   model_nm  category
 3   fwver     category
 4   errtype   int8    
 5   errcode   category
dtypes: category(3), int16(1), int64(1), int8(1)
memory usage: 236.9 MB


In [151]:
test_q.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747972 entries, 0 to 747971
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   time        747972 non-null  int64   
 1   user_id     747972 non-null  uint16  
 2   fwver       725208 non-null  category
 3   quality_0   641388 non-null  float32 
 4   quality_1   747961 non-null  category
 5   quality_2   726857 non-null  float32 
 6   quality_3   747972 non-null  int8    
 7   quality_4   747972 non-null  int8    
 8   quality_5   747928 non-null  category
 9   quality_6   747972 non-null  int16   
 10  quality_7   747972 non-null  category
 11  quality_8   747972 non-null  category
 12  quality_9   747972 non-null  category
 13  quality_10  747972 non-null  category
 14  quality_11  747972 non-null  int8    
 15  quality_12  747972 non-null  int8    
dtypes: category(7), float32(2), int16(1), int64(1), int8(4), uint16(1)
memory usage: 25.4 MB


In [152]:
test_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16532648 entries, 0 to 16532647
Data columns (total 6 columns):
 #   Column    Dtype   
---  ------    -----   
 0   user_id   uint16  
 1   time      int64   
 2   model_nm  category
 3   fwver     category
 4   errtype   int8    
 5   errcode   category
dtypes: category(3), int64(1), int8(1), uint16(1)
memory usage: 236.6 MB


In [153]:
def preprocessing_problem(df, object_='binary'):
    """
    definition:
    train_problem 테이블을 받아서 target 값으로 변환
    1. {0, 1}의 binary로 변환
    2. {0 ~ n}의 multiclass로 변환
    """
    
    # 10001부터 24999까지의 index를 만들어줍니다.
    user_id_idx = np.array(range(10000, 25000, 1))
    
    # train_new_p라는 새로운 df를 만들고 index는 위에서 만든 user_id_idx 로 지정해줍니다.
    new_p = pd.DataFrame(index = user_id_idx)
    new_p['target'] = 0
    
    if object_ == 'binary':
        new_p.iloc[df.user_id.unique()-10000] = 1
        new_p = new_p.reset_index()
        new_p.rename({'index':'user_id'}, axis=1, inplace=True)
        
    elif object_ == 'multi':
        # multi는 count()로 집계를 해줍니다.
        new_p['target'] = df.groupby('user_id')['time'].count()
        new_p = new_p.fillna(0)
        new_p = new_p.reset_index()
        new_p.rename({'index':'user_id'}, axis=1, inplace=True)
        
    return new_p

In [154]:
train_b_p = preprocessing_problem(train_p, 'binary')
train_m_p = preprocessing_problem(train_p, 'multi')

In [155]:
def preprocessing_quality(df):
    """
    definition:
    EDA를 통해 알아낸 정보로 train_q, test_q를 정리해서 내뿜어줍니다.
    1. qaulity_3, quality_4 를 drop 합니다.(단일 value)
    2. qaulity_k 변수들을 정수로 encoding 합니다.
    """
    # 먼저 3, 4번을 drop 합니다.
    df.drop(['quality_3', 'quality_4'], axis=1, inplace=True)
    
    # qual 변수만 할당해주고, 정수로 형변환 해줍니다.
    columns = train_q.columns[train_q.columns.str.contains('quality')]
    # for문을 통해 각 column을 반복 작업해줍니다.
    for col in columns:
        try:
            if df[col].dtype == 'float32': # 기존에 float은 패스
                df[col] = df[col].fillna(-2)
            elif df[col].dtype == 'int8' or df[col].dtype == 'int16': # 기존에 int도 패스
                df[col] = df[col].fillna(-2)
                #print(col)
            else:
                df[col] = df[col].astype('object')
                # nan값이 있으면 float으로 갈 수 없으니 '-2' 으로 채워줍니다.
                df[col] = df[col].fillna('-2')
                df[col] = df[col].apply(lambda x: x.replace(',' , ''))
                df[col] = df[col].astype(np.float32)
        except:
            pass
        
    # fwver 에서 null 값이 꽤 있습니다. missing으로 채우겠습니다.
    df.fwver = df.fwver.astype('object')
    df.fwver = df.fwver.fillna('missing')
    df.fwver = df.fwver.astype('category')
        
    return df

In [156]:
train_q = preprocessing_quality(train_q)
test_q = preprocessing_quality(test_q)

In [157]:
def preprocessing_fwver(df):
    """
    definition:
    별건 아니고, e-set에 fwver 변수에서 '10' 이라는 값이 있는데, 이게 errtype이랑 겹쳐요.
    그래서 10을 -> 8.5.2 으로 바꿔줄 겁니다.
    굳이 이렇게 바꾸는 이유는, 해당 fw가 8.5.3버전과 같은 model_nm을 공유하기 때문입니다.
    """
    df.fwver = df.fwver.replace('10', '8.5.2')
        
    return df

In [158]:
train_e = preprocessing_fwver(train_e)
test_e = preprocessing_fwver(test_e)

In [159]:
def make_datetime(df):
    """
    definition:
    'time' column이 str로 되어 있으니, datetime으로 바꿔주는 함수입니다.
    다만 'time'양식이 pandas함수에 적용이 안되니, 강제로 슬라이싱해서 만들어줘야 합니다.
    
    """
    
    df['year'] = df['time'].apply(lambda x: str(x)[:4])
    df['month'] = df['time'].apply(lambda x: str(x)[4:6])
    df['day'] = df['time'].apply(lambda x: str(x)[6:8])
    df['hour'] = df['time'].apply(lambda x: str(x)[8:10])
    df['minute'] = '00' # minute을 넣어주지 않으면 datetime이 완성이 안되니, 00으로 넣어줍니다.
    
    df['time'] = pd.to_datetime(df.year + df.month + df.day + df.hour + df.minute)
    
    return df

In [160]:
train_p = make_datetime(train_p)
train_q = make_datetime(train_q)
test_q = make_datetime(test_q)
train_e = make_datetime(train_e)
test_e = make_datetime(test_e)

In [161]:
# groupby 연산에서 왜 문제가 생기나 했더니 category로 되어 있어서였습니다.

train_e['errtype'] = train_e.errtype.astype('object')
test_e['errtype'] = test_e.errtype.astype('object')

train_e['errcode'] = train_e.errcode.astype('object')
test_e['errcode'] = test_e.errcode.astype('object')

In [162]:
# fwver도 object로 잡아줍니다.

train_q.fwver = train_q.fwver.astype('object')
test_q.fwver = test_q.fwver.astype('object')

# EDA feature Collection 2

### errcode 마무리

In [21]:
temp = train_e[['errtype', 'errcode']].drop_duplicates()
count = {}

for i in sorted(temp.errtype.unique()):
    count[i] = (len(temp[temp.errtype == i]['errcode'].unique()))

In [22]:
err_type_user_counts = []
for i in sorted(train_e.errtype.unique()):
    err_type_user_counts.append(train_e[train_e.errtype == i].groupby('user_id').count().shape[0])

train_p['target'] = 1
temp = pd.merge(train_e, train_p, how='left', on=['user_id'])
temp2 = temp[temp.target == 1]

err_type_user_counts_origin = []
for i in sorted(temp2.errtype.unique()):
    err_type_user_counts_origin.append(temp2[temp2.errtype == i].groupby('user_id').count().shape[0])

In [23]:
etuc_1 = np.array(err_type_user_counts)
etuc_2 = np.array(err_type_user_counts_origin)

In [24]:
errcode_df = pd.DataFrame()
errcode_df['etuc_1'] = etuc_1
errcode_df['etuc_2'] = etuc_2
errcode_df['div'] = ((etuc_2 / etuc_1) * 100).round(2)
errcode_df['unique'] = count.values()

In [25]:
errcode_df = errcode_df.set_index([sorted(train_e.errtype.unique())])

In [26]:
errcode_df['perf'] = np.nan

In [27]:
errcode_df.loc[errcode_df.unique == 1, 'perf'] = 'pass'
errcode_df.loc[errcode_df.index == 31, 'perf'] = 'reject'
errcode_df.loc[errcode_df.index == 41, 'perf'] = 'reject'
errcode_df.loc[errcode_df.index == 17, 'perf'] = 'reject'
errcode_df.loc[errcode_df.index == 23, 'perf'] = 'adopt'
errcode_df.loc[errcode_df.index == 33, 'perf'] = 'adopt'
errcode_df.loc[errcode_df.index == 40, 'perf'] = 'reject'
errcode_df.loc[errcode_df.index == 14, 'perf'] = 'reject'
errcode_df.loc[errcode_df.index == 37, 'perf'] = 'reject'
errcode_df.loc[errcode_df.index == 7, 'perf'] = 'reject'
errcode_df.loc[errcode_df.index == 4, 'perf'] = 'reject'
errcode_df.loc[errcode_df.index == 6, 'perf'] = 'reject'
errcode_df.loc[errcode_df.index == 34, 'perf'] = 'adopt'
errcode_df.loc[errcode_df.index == 5, 'perf'] = 'reject'
errcode_df.loc[errcode_df.index == 42, 'perf'] = 'reject'
errcode_df.loc[errcode_df.index == 38, 'perf'] = 'reject'
errcode_df.loc[errcode_df.index == 32, 'perf'] = 'hold' # model 성능은 올라갔으나, 리더보드 하락
errcode_df.loc[errcode_df.index == 3, 'perf'] = 'hold'

In [28]:
errcode_df[errcode_df.perf.isnull()]

etuc_1  etuc_2    div  unique perf
1      950     242  25.47       8  NaN
2      390     168  43.08       2  NaN
8       64      13  20.31       3  NaN
9       63      38  60.32       9  NaN
25     420     360  85.71       9  NaN
30     375     260  69.33       5  NaN
39     766     405  52.87       2  NaN

In [364]:
errcode_df.sort_values('etuc_1')

etuc_1  etuc_2    div  unique    perf
9       63      38  60.32       9     NaN
8       64      13  20.31       3     NaN
30     375     260  69.33       5     NaN
21     381     316  82.94       1    pass
2      390     168  43.08       2     NaN
19     419     346  82.58       1    pass
25     420     360  85.71       9     NaN
39     766     405  52.87       2     NaN
1      950     242  25.47       8     NaN
20    1546    1333  86.22       1    pass
3     1679     465  27.70       3    hold
18    1768    1521  86.03       1    pass
38    1822     927  50.88    2653  reject
32    1933     927  47.96      55    hold
35    2039    1186  58.17       1    pass
27    2267     958  42.26       1    pass
28    2275     959  42.15       1    pass
10    2360    1134  48.05       1    pass
24    2519    1166  46.29       1    pass
13    3711    1667  44.92       1    pass
34    3733    1963  52.59       6   adopt
17    5747    2498  43.47       5  reject
41    6142    2596  42.27       1  reject
42    6382    2490  39.02       2  reject
6     6467    2662  41.16       2  reject
4     6881    2298  33.40       2  reject
7     7648    3093  40.44       2  reject
14    7891    3224  40.86       3  reject
37    8434    3259  38.64       2  reject
36    8435    3262  38.67       1    pass
23   10046    3920  39.02       8   adopt
22   10281    3975  38.66       1    pass
40   10462    4052  38.73       2  reject
5    11010    4028  36.58      65  reject
33   11074    4138  37.37       3   adopt
31   11091    4153  37.44       2  reject
26   13026    4632  35.56       1    pass
16   14431    4977  34.49       1    pass
15   14545    4978  34.22       1    pass
11   14767    4971  33.66       1    pass
12   14799    4974  33.61       1    pass

이 친구들은 도전하지 않겠습니다.

### Quality 다시 살펴보기

가설 세우기.

1. fwver별로 퀄리티 평균을 구해서 fwver별로 차이를 구해준다면. 제일 처음 것과 제일 마지막 것의 차이를 주는 것. 1개만 쓰면 0, 음수가 될 수도 있고, 양수가 될 수도 있고.
2. 단순히 q로그가 몇 번 있었는지를 기록했는데, 정지훈 선생의 가설 대로 0이 default 값이라면, 0이 아닌 값이 몇 번 일어났는가도 중요하지 않을까.
3. 

#### quality log 살펴보기

In [29]:
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    print(i,'번째 nunique: ' ,train_q['quality_{}'.format(i)].nunique())

0 번째 nunique:  754
1 번째 nunique:  31
2 번째 nunique:  799
5 번째 nunique:  4745
6 번째 nunique:  549
7 번째 nunique:  884
8 번째 nunique:  42
9 번째 nunique:  523
10 번째 nunique:  4200
11 번째 nunique:  12
12 번째 nunique:  14


In [30]:
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    print(i,'번째 nunique: ' ,test_q['quality_{}'.format(i)].nunique())

0 번째 nunique:  541
1 번째 nunique:  35
2 번째 nunique:  597
5 번째 nunique:  3995
6 번째 nunique:  528
7 번째 nunique:  839
8 번째 nunique:  44
9 번째 nunique:  431
10 번째 nunique:  3515
11 번째 nunique:  13
12 번째 nunique:  17


In [31]:
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    print(train_q['quality_{}'.format(i)].value_counts().head(10))

 0.0    542790
-2.0    144432
-1.0    130828
 1.0      2097
 2.0      1252
 3.0       518
 4.0       410
 5.0       385
 6.0       358
 7.0       314
Name: quality_0, dtype: int64
 0    670270
-1    153649
 1      2567
 2      1140
 3       391
 4       202
 5       134
 6        71
 7        50
 8        35
Name: quality_1, dtype: int64
 0.0    632469
-1.0    144392
-2.0     40113
 1.0      2937
 2.0      1073
 3.0       580
 4.0       455
 5.0       427
 6.0       386
 7.0       337
Name: quality_2, dtype: int64
 0.0    428096
-1.0    153354
 1.0     56171
 2.0     35978
 3.0     21596
 4.0     11204
 5.0      8536
 6.0      6836
 7.0      5506
 8.0      4495
Name: quality_5, dtype: int64
 0      662217
-1      153531
 600      1852
 1        1347
 5         644
 2         603
 3         475
 6         445
 4         408
 10        320
Name: quality_6, dtype: int64
0.0     757788
1.0       6768
2.0       3648
5.0       3456
6.0       2688
3.0       2568
4.0       2124
10.0      1632


In [32]:
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    print(test_q['quality_{}'.format(i)].value_counts().head(10))

 0.0    505710
-1.0    127001
-2.0    106584
 1.0      2319
 2.0      1567
 3.0       739
 4.0       454
 5.0       290
 6.0       212
 8.0       158
Name: quality_0, dtype: int64
 0.0    601265
-1.0    141905
 1.0      2573
 2.0      1119
 3.0       451
 4.0       201
 5.0       130
 6.0       101
 7.0        58
 8.0        37
Name: quality_1, dtype: int64
 0.0    578838
-1.0    136864
-2.0     21115
 1.0      3690
 2.0      1629
 3.0       962
 4.0       625
 5.0       391
 6.0       295
 7.0       203
Name: quality_2, dtype: int64
 0.0    399032
-1.0    141788
 1.0     52431
 2.0     33814
 3.0     20164
 4.0     10797
 5.0      8320
 6.0      6500
 7.0      4821
 8.0      3983
Name: quality_5, dtype: int64
 0      594594
-1      141866
 600      1848
 1        1206
 5         548
 2         521
 3         418
 4         400
 6         393
 10        309
Name: quality_6, dtype: int64
0.0     682584
1.0       6408
5.0       3612
2.0       2988
6.0       2400
3.0       2220
4.0       

7, 8, 9, 10, 12 는 음수 값이 없습니다. 0, 1, 2, 5, 6, 11 은 음수 값이 존재합니다.

In [33]:
train_q_some = train_q[['user_id', 'quality_7', 'quality_8', 'quality_9', 'quality_10', 'quality_12']]
test_q_some = test_q[['user_id', 'quality_7', 'quality_8', 'quality_9', 'quality_10', 'quality_12']]

In [34]:
train_q_some

user_id  quality_7  quality_8  quality_9  quality_10  quality_12
0         10000        0.0        0.0        0.0         4.0           0
1         10000        0.0        0.0        0.0         4.0           0
2         10000        0.0        0.0        0.0         4.0           0
3         10000        0.0        0.0        0.0         4.0           0
4         10000        0.0        0.0        0.0         4.0           0
...         ...        ...        ...        ...         ...         ...
828619    24997        0.0        0.0        0.0        17.0           0
828620    24997        0.0        0.0        0.0        17.0           0
828621    24997        0.0        0.0        0.0        17.0           0
828622    24997        0.0        0.0        0.0        17.0           0
828623    24997        0.0        0.0        0.0        17.0           0

[828624 rows x 6 columns]

#### 몇 번이나 연속해서 quality가 기록되었는가?

In [111]:
temp = train_q.groupby('user_id').count()['time']

In [122]:
temp.loc[:] = np.nan

In [123]:
temp

user_id
10000   NaN
10002   NaN
10004   NaN
10005   NaN
10006   NaN
         ..
24990   NaN
24992   NaN
24993   NaN
24995   NaN
24997   NaN
Name: time, Length: 8281, dtype: float64

In [207]:
temp_train = train_q.groupby('user_id').count()['time']
user_quality_log_gap = {}
for user in train_q.user_id.unique():
    time = train_q[train_q.user_id == user].time.drop_duplicates().values
    time_sum = 0
    if len(time) < 1:
        pass
    else:
        for i in range(len(time) - 1):
            time_sum += time[i] - time[i+1]
            
        temp_train[user] = time_sum/60000000000
        
temp_train = temp_train.rename('time_gap')

In [208]:
temp_test = test_q.groupby('user_id').count()['time']
user_quality_log_gap = {}
for user in test_q.user_id.unique():
    time = test_q[test_q.user_id == user].time.drop_duplicates().values
    time_sum = 0
    if len(time) < 1:
        pass
    else:
        for i in range(len(time) - 1):
            time_sum += time[i] - time[i+1]
            
        temp_test[user] = time_sum/60000000000
        
temp_test = temp_test.rename('time_gap')

In [209]:
temp_train

user_id
10000    -2160
10002   -36960
10004    -2220
10005     -120
10006   -24840
         ...  
24990   -18120
24992    -1440
24993   -42180
24995     -240
24997   -21840
Name: time_gap, Length: 8281, dtype: int64

In [210]:
temp_test

user_id
30000        0
30001   -13440
30002   -25320
30003   -12960
30004    -7200
         ...  
44990   -28380
44993   -35700
44994        0
44996   -33840
44997   -32940
Name: time_gap, Length: 8268, dtype: int64

In [97]:
time = train_q[train_q.user_id == 10019].time.drop_duplicates().values

In [101]:
time[2]

numpy.datetime64('2020-11-04T21:00:00.000000000')

In [78]:
train_p[train_p.user_id == 10019]

user_id                time  year month day hour minute  target
4158    10019 2020-11-24 11:00:00  2020    11  24   11     00       1

In [80]:
train_e[train_e.user_id == 10019].time.drop_duplicates()

23826   2020-11-01 06:00:00
23831   2020-11-01 10:00:00
23836   2020-11-01 11:00:00
23837   2020-11-01 18:00:00
23842   2020-11-02 06:00:00
                ...        
24649   2020-11-29 18:00:00
24654   2020-11-30 07:00:00
24659   2020-11-30 10:00:00
24663   2020-11-30 12:00:00
24668   2020-11-30 18:00:00
Name: time, Length: 153, dtype: datetime64[ns]

In [240]:
train_q.time[0] + pd.Timedelta(hours=2) == train_q.time[12]

False

In [238]:
train_q.time[12]

Timestamp('2020-11-30 21:00:00')

In [268]:
train_origin_time = train_q[['user_id', 'time']].drop_duplicates()
time_continue = []

for i in range(train_origin_time.shape[0] - 1):
    if train_origin_time.user_id.iloc[i] == train_origin_time.user_id.iloc[i+1]:
        if train_origin_time.time.iloc[i] + pd.Timedelta(hours=2) == train_origin_time.time.iloc[i+1]:
            time_continue.append(1)
        else:
            time_continue.append(0)
    else:
        time_continue.append(0)
        
train_origin_time['t_continue'] = time_continue + [0]

train_qual_conti = train_origin_time.groupby(['user_id'])['t_continue'].sum()

In [269]:
test_origin_time = test_q[['user_id', 'time']].drop_duplicates()
time_continue = []

for i in range(test_origin_time.shape[0] - 1):
    if test_origin_time.user_id.iloc[i] == test_origin_time.user_id.iloc[i+1]:
        if test_origin_time.time.iloc[i] + pd.Timedelta(hours=2) == test_origin_time.time.iloc[i+1]:
            time_continue.append(1)
        else:
            time_continue.append(0)
    else:
        time_continue.append(0)
        
test_origin_time['t_continue'] = time_continue + [0]

test_qual_conti = test_origin_time.groupby(['user_id'])['t_continue'].sum()

In [270]:
train_qual_conti

user_id
10000    0
10002    1
10004    0
10005    1
10006    0
        ..
24990    0
24992    0
24993    0
24995    0
24997    0
Name: t_continue, Length: 8281, dtype: int64

In [271]:
test_qual_conti

user_id
30000    0
30001    0
30002    3
30003    1
30004    0
        ..
44990    0
44993    0
44994    0
44996    1
44997    0
Name: t_continue, Length: 8268, dtype: int64

#### quality 별로 몇 번이나 기록되었는가?

In [35]:
train_q_temp = train_q.copy()
test_q_temp = test_q.copy()

count를 위해서 0 값을 전부 nan으로 만들겠습니다.

In [36]:
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] == 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] == 0, 'quality_{}'.format(i)] = np.nan

In [37]:
train_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)
test_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)

In [38]:
train_qual_counts = train_q_temp.groupby('user_id').count()
test_qual_counts = test_q_temp.groupby('user_id').count()

In [39]:
train_qual_counts.columns = ['q_c_0', 'q_c_1', 'q_c_2', 'q_c_5', 'q_c_6',
                             'q_c_7', 'q_c_8', 'q_c_9', 'q_c_10', 'q_c_11', 'q_c_12']
test_qual_counts.columns = ['q_c_0', 'q_c_1', 'q_c_2', 'q_c_5', 'q_c_6',
                             'q_c_7', 'q_c_8', 'q_c_9', 'q_c_10', 'q_c_11', 'q_c_12']

리더보드는 올랐습니다. 아주 미세하게.

#### 0을 빼고 전부 1로 바꿔버리는 겁니다. 그리고 옆으로 스택해주는 것이죠. 몰라요 그냥 해볼래요.

In [35]:
train_q_temp = train_q.copy()
test_q_temp = test_q.copy()

In [36]:
train_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)
test_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)

In [26]:
# 0 값만 count를 위해서 음수와 양수를 전부 nan으로 만들겠습니다.

for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] < 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] < 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] > 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] > 0, 'quality_{}'.format(i)] = np.nan

In [29]:
train_qual_zeroCount = train_q_temp.groupby('user_id').count()
test_qual_zeroCount = test_q_temp.groupby('user_id').count()

In [42]:
new_columns = ['q_z_c_0', 'q_z_c_1', 'q_z_c_2', 'q_z_c_5', 'q_z_c_6', 'q_z_c_7', 'q_z_c_8', 'q_z_c_9', 'q_z_c_10','q_z_c_11', 'q_z_c_12']

train_qual_zeroCount.columns = new_columns
test_qual_zeroCount.columns = new_columns

In [35]:
train_q_temp = train_q.copy()
test_q_temp = test_q.copy()

In [36]:
train_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)
test_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)

In [32]:
# 양수 값만 count를 위해서 음수와 0을 전부 nan으로 만들겠습니다.

for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] <= 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] <= 0, 'quality_{}'.format(i)] = np.nan

In [34]:
train_qual_posiCount = train_q_temp.groupby('user_id').count()
test_qual_posiCount = test_q_temp.groupby('user_id').count()

In [43]:
new_columns = ['q_p_c_0', 'q_p_c_1', 'q_p_c_2', 'q_p_c_5', 'q_p_c_6', 'q_p_c_7', 'q_p_c_8', 'q_p_c_9', 'q_p_c_10','q_p_c_11', 'q_p_c_12']

train_qual_posiCount.columns = new_columns
test_qual_posiCount.columns = new_columns

In [35]:
train_q_temp = train_q.copy()
test_q_temp = test_q.copy()

In [36]:
train_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)
test_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)

In [37]:
# 음수 값만 count를 위해서 음수와 0을 전부 nan으로 만들겠습니다.

for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] >= 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] >= 0, 'quality_{}'.format(i)] = np.nan

In [40]:
train_qual_negaCount = train_q_temp.groupby('user_id').count()
test_qual_negaCount = test_q_temp.groupby('user_id').count()

In [44]:
new_columns = ['q_n_c_0', 'q_n_c_1', 'q_n_c_2', 'q_n_c_5', 'q_n_c_6', 'q_n_c_7', 'q_n_c_8', 'q_n_c_9', 'q_n_c_10','q_n_c_11', 'q_n_c_12']

train_qual_negaCount.columns = new_columns
test_qual_negaCount.columns = new_columns

#### 음수만 카운트, 0만 카운트, 양수만 카운트 해서 언스택 해주겠습니다.

In [21]:
train_q_temp = train_q.copy()
test_q_temp = test_q.copy()

In [22]:
train_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)
test_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)

In [301]:
qual_columns = ['quality_0', 'quality_1', 'quality_2', 'quality_5', 'quality_6', 'quality_7', 'quality_8', 'quality_9', 'quality_10','quality_11', 'quality_12']

for col in qual_columns:
    train_q_temp[col] = train_q_temp[col].apply(lambda x: 1 if x != 0 else x)
    
for col in qual_columns:
    test_q_temp[col] = test_q_temp[col].apply(lambda x: 1 if x != 0 else x)

In [316]:
train_qual_onehot = train_q_temp.groupby('user_id').max()
test_qual_onehot = test_q_temp.groupby('user_id').max()

In [317]:
new_columns = ['q_o_h_0', 'q_o_h_1', 'q_o_h_2', 'q_o_h_5', 'q_o_h_6', 'q_o_h_7', 'q_o_h_8', 'q_o_h_9', 'q_o_h_10','q_o_h_11', 'q_o_h_12']

train_qual_onehot.columns = new_columns
test_qual_onehot.columns = new_columns

#### 유저당 등장한 유니크한 errtype, errcode 은 총 몇 개인가?

In [85]:
train_e[['user_id', 'errtype']].drop_duplicates().groupby('user_id').count()
test_e[['user_id', 'errtype']].drop_duplicates().groupby('user_id').count()

errtype
user_id         
10000         14
10001         26
10002         16
10003         18
10004         18
...          ...
24995         17
24996          4
24997         21
24998         14
24999         12

[15000 rows x 1 columns]

In [182]:
train_errtype_uniqueCount = train_e[['user_id', 'errtype']].drop_duplicates().groupby('user_id').count()
test_errtype_uniqueCount = test_e[['user_id', 'errtype']].drop_duplicates().groupby('user_id').count()
train_errtype_uniqueCount.columns, test_errtype_uniqueCount.columns = ['errtype_uc'], ['errtype_uc']

In [87]:
train_e[['user_id', 'errcode']].drop_duplicates().groupby('user_id').count()

errcode
user_id         
10000          3
10001         12
10002          4
10003         13
10004         12
...          ...
24995         10
24996          2
24997         13
24998          8
24999          5

[15000 rows x 1 columns]

In [183]:
train_errcode_uniqueCount = train_e[['user_id', 'errcode']].drop_duplicates().groupby('user_id').count()
test_errcode_uniqueCount = test_e[['user_id', 'errcode']].drop_duplicates().groupby('user_id').count()
train_errcode_uniqueCount.columns, test_errcode_uniqueCount.columns = ['errcode_uc'], ['errcode_uc']

In [79]:
train_e.groupby('user_id')['errcode'].count()

user_id
10000     317
10001    2365
10002     306
10003     306
10004     777
         ... 
24995     194
24996       4
24997     826
24998     155
24999     570
Name: errcode, Length: 15000, dtype: int64

#### fwver별로 quality 살펴보기

In [181]:
train_b_p[train_b_p.user_id == 10229]

user_id  target
229    10229       0

In [288]:
train_q

time  user_id       fwver  quality_0  quality_1  \
0      2020-11-29 09:00:00    10000  05.15.2138        0.0          0   
1      2020-11-29 09:00:00    10000  05.15.2138        0.0          0   
2      2020-11-29 09:00:00    10000  05.15.2138        0.0          0   
3      2020-11-29 09:00:00    10000  05.15.2138        0.0          0   
4      2020-11-29 09:00:00    10000  05.15.2138        0.0          0   
...                    ...      ...         ...        ...        ...   
828619 2020-11-24 03:00:00    24997  04.22.1778        0.0          0   
828620 2020-11-24 03:00:00    24997  04.22.1778        0.0          0   
828621 2020-11-24 03:00:00    24997  04.22.1778        0.0          0   
828622 2020-11-24 03:00:00    24997  04.22.1778        0.0          0   
828623 2020-11-24 03:00:00    24997  04.22.1778        0.0          0   

        quality_2  quality_5  quality_6  quality_7  quality_8  quality_9  \
0             0.0        0.0          0        0.0        0.0        0.0   
1             0.0        0.0          0        0.0        0.0        0.0   
2             0.0        0.0          0        0.0        0.0        0.0   
3             0.0        0.0          0        0.0        0.0        0.0   
4             0.0        0.0          0        0.0        0.0        0.0   
...           ...        ...        ...        ...        ...        ...   
828619        0.0        1.0          0        0.0        0.0        0.0   
828620        0.0        0.0          0        0.0        0.0        0.0   
828621        0.0        3.0          0        0.0        0.0        0.0   
828622        0.0        0.0          0        0.0        0.0        0.0   
828623        0.0        9.0          0        0.0        0.0        0.0   

        quality_10  quality_11  quality_12  year month day hour minute  
0              4.0           0           0  2020    11  29   09     00  
1              4.0           0           0  2020    11  29   09     00  
2              4.0           0           0  2020    11  29   09     00  
3              4.0           0           0  2020    11  29   09     00  
4              4.0           0           0  2020    11  29   09     00  
...            ...         ...         ...   ...   ...  ..  ...    ...  
828619        17.0           0           0  2020    11  24   03     00  
828620        17.0           0           0  2020    11  24   03     00  
828621        17.0           0           0  2020    11  24   03     00  
828622        17.0           0           0  2020    11  24   03     00  
828623        17.0           0           0  2020    11  24   03     00  

[828624 rows x 19 columns]

In [287]:
train_q.groupby(['user_id', 'fwver']).mean().reset_index()[100:200]

user_id       fwver  quality_0  quality_1   quality_2   quality_5  \
100    10152  04.16.3553   0.000000   0.041667    0.000000    4.250000   
101    10153  04.33.1261  -0.166667  -0.166667   -0.166667    0.083333   
102    10154  04.33.1185  -0.333333  -0.333333   -0.333333   -0.083333   
103    10154  04.33.1261  -0.416667  -0.416667   -0.416667   23.041666   
104    10158  04.16.3553   0.000000   0.000000    0.000000    0.416667   
105    10160  04.16.3553   0.000000   0.000000    0.000000    0.250000   
106    10161  04.16.3553  -0.500000  -0.500000   -0.500000   -0.166667   
107    10163  04.22.1750   0.000000   0.000000    0.000000    0.567901   
108    10163  04.22.1778  -0.229167  -0.229167   -0.229167    0.208333   
109    10165  04.16.3553  -0.305556  -0.277778   -0.305556    0.500000   
110    10166  04.33.1185  -0.125000  -0.125000   -0.125000    0.125000   
111    10166  04.33.1261  -0.208333  -0.208333   -0.208333  137.902771   
112    10167  03.11.1167  -2.000000  -0.267628   -0.275641    7.028846   
113    10168  03.11.1167  -2.000000   0.000000    0.000000    0.083333   
114    10171  03.11.1167  -2.000000   0.000000    0.000000    2.505952   
115    10173  04.16.3553   0.000000   0.000000    0.000000    0.208333   
116    10173  04.33.1261  -0.083333  -0.062500   -0.083333    0.625000   
117    10174  04.33.1185  -0.416667  -0.416667   -0.416667   -0.083333   
118    10174  04.33.1261  -0.041667  -0.041667   -0.041667    0.041667   
119    10175  04.16.3553   0.000000   0.000000    0.000000  213.375000   
120    10178  04.22.1750  -0.250000  -0.250000   -0.250000  157.766663   
121    10184  05.15.2138  -0.333333  -0.333333   -0.333333    0.541667   
122    10187  04.16.3553  -0.166667  -0.166667   -0.166667    0.291667   
123    10188  04.16.3553  -0.083333  -0.083333   -0.083333    0.166667   
124    10189  04.22.1750  -0.375000  -0.375000   -0.375000   -0.125000   
125    10191  05.15.2138   0.000000   0.000000    0.000000    0.250000   
126    10192  04.33.1261  -0.333333  -0.333333   -0.333333    0.250000   
127    10193  05.15.2138  -0.125000  -0.208333   -0.166667   -0.041667   
128    10194  04.16.3553  -0.083333  -0.083333   -0.083333    0.183333   
129    10195  05.15.2138  -0.093434   0.068182   -0.098485   -0.068182   
130    10197  04.16.3553  -0.166667  -0.166667   -0.166667    0.250000   
131    10198  04.33.1185   0.000000   0.000000    0.000000    0.583333   
132    10198  04.33.1261   0.000000   0.000000    0.000000    0.166667   
133    10199  04.22.1778  -0.250000  -0.250000   -0.250000    0.000000   
134    10201  04.22.1750  -0.027778  -0.027778   -0.027778  481.111115   
135    10203  05.15.2138  -0.250000  -0.250000   -0.250000   -0.041667   
136    10205  05.15.2138  -0.111111  -0.111111   -0.111111    0.138889   
137    10206  03.11.1167  -2.000000  -0.183333   -0.183333   -0.066667   
138    10207  03.11.1167  -2.000000  -0.150000   -0.150000    0.200000   
139    10208  04.22.1750   0.000000   0.000000    0.000000    0.375000   
140    10209  04.16.3553  -0.083333  -0.055556   -0.083333    0.277778   
141    10210  04.33.1185   0.000000   0.000000    0.000000    0.291667   
142    10210  04.33.1261  -0.222222  -0.222222   -0.222222    0.027778   
143    10213  04.22.1750  -0.011905  -0.011905   -0.011905    0.452381   
144    10213  04.22.1778  -0.055556  -0.055556   -0.055556    0.194444   
145    10214  04.16.3553  -0.166667  -0.166667   -0.166667    0.000000   
146    10215  04.16.3553  -0.416667  -0.416667   -0.416667   -0.166667   
147    10217  03.11.1167  -2.000000  -0.232639   -0.267361   -0.256944   
148    10218  04.22.1750  -0.166667  -0.041667   -0.166667   -0.041667   
149    10218  04.22.1778  -0.583333  -0.583333   -0.583333   -0.333333   
150    10219  04.33.1185  -0.305556  -0.305556   -0.305556   -0.027778   
151    10219  04.33.1261  -0.523810  -0.523810   -0.523810    0.023810   
152    10222  04.33.1261   0.000000   0.000000    0.000000    0.333333   
153    10

In [328]:
train_q.groupby('user_id').mean()

quality_0  quality_1  quality_2  quality_5  quality_6  quality_7  \
user_id                                                                     
10000     0.000000   0.000000   0.000000   0.500000   0.000000   0.000000   
10002     0.000000  -0.020833  -0.010417   0.302083   0.458333   5.750000   
10004    -0.083333  -0.083333  -0.083333   0.083333   3.541667  43.500000   
10005    -0.416667  -0.416667  -0.416667   0.000000   1.083333  18.000000   
10006     0.000000   0.000000   0.000000   0.333333   0.111111   1.333333   
...            ...        ...        ...        ...        ...        ...   
24990    -2.000000  -0.333333  -0.333333  -0.250000  -0.333333   0.000000   
24992    -0.333333  -0.333333  -0.333333   0.194444  -0.333333   0.000000   
24993    -0.291667  -0.291667  -0.291667   0.041667  -0.291667   0.000000   
24995    -0.416667  -0.416667  -0.416667  -0.208333  -0.416667   0.000000   
24997    -0.166667  -0.166667  -0.166667   0.625000  -0.166667   0.000000   

         quality_8  quality_9  quality_10  quality_11  quality_12  
user_id                                                            
10000          0.0      0.000    6.000000    0.000000         0.0  
10002          0.0      0.125    3.875000   -0.020833         0.0  
10004          0.0      0.000    2.000000   -0.083333         0.0  
10005          0.0      0.000    5.000000   -0.416667         0.0  
10006          0.0      0.000    4.000000    0.000000         0.0  
...            ...        ...         ...         ...         ...  
24990          0.0      0.000    1.000000   -0.333333         0.0  
24992          0.0      0.000    6.333333   -0.333333         0.0  
24993          0.0      0.000    4.000000   -0.291667         0.0  
24995          0.0      0.000    2.500000   -0.416667         0.0  
24997          0.0      0.000    9.500000   -0.166667         0.0  

[8281 rows x 11 columns]

In [329]:
train_q.groupby('user_id').std()

quality_0  quality_1  quality_2  quality_5  quality_6  quality_7  \
user_id                                                                     
10000     0.000000   0.000000   0.000000   1.793709   0.000000   0.000000   
10002     0.251312   0.143576   0.177396   1.037404   4.495417  14.547924   
10004     0.282330   0.282330   0.282330   0.583592  17.778784  44.435591   
10005     0.503610   0.503610   0.503610   1.179536   5.867498  18.387141   
10006     0.000000   0.000000   0.000000   1.121224   0.666667   1.912366   
...            ...        ...        ...        ...        ...        ...   
24990     0.000000   0.481543   0.481543   0.607919   0.481543   0.000000   
24992     0.478091   0.478091   0.478091   1.879505   0.478091   0.000000   
24993     0.464306   0.464306   0.464306   1.334465   0.464306   0.000000   
24995     0.503610   0.503610   0.503610   0.977093   0.503610   0.000000   
24997     0.380693   0.380693   0.380693   2.060234   0.380693   0.000000   

         quality_8  quality_9  quality_10  quality_11  quality_12  
user_id                                                            
10000          0.0   0.000000    2.043016    0.000000         0.0  
10002          0.0   0.332455    2.327185    0.143576         0.0  
10004          0.0   0.000000    1.021508    0.282330         0.0  
10005          0.0   0.000000    1.021508    0.503610         0.0  
10006          0.0   0.000000    0.000000    0.000000         0.0  
...            ...        ...         ...         ...         ...  
24990          0.0   0.000000    0.000000    0.481543         0.0  
24992          0.0   0.000000    2.908117    0.478091         0.0  
24993          0.0   0.000000    1.021508    0.464306         0.0  
24995          0.0   0.000000    0.510754    0.503610         0.0  
24997          0.0   0.000000    7.661309    0.380693         0.0  

[8281 rows x 11 columns]

In [332]:
train_qual_mean = train_q.groupby(['user_id']).mean()
test_qual_mean = test_q.groupby(['user_id']).mean()

new_columns = ['q_mean_0', 'q_mean_1', 'q_mean_2', 'q_mean_5', 'q_mean_6', 'q_mean_7', 'q_mean_8', 'q_mean_9', 'q_mean_10','q_mean_11', 'q_mean_12']

train_qual_mean.columns = new_columns
test_qual_mean.columns = new_columns

## q에 한 번도 등장하지 않은 user

In [455]:
origin_q_train_user = list(train_q.user_id.unique())
train_all_user = list(np.arange(10000, 25000, 1))

In [456]:
origin_q_test_user = list(test_q.user_id.unique())
test_all_user = list(np.arange(30000, 49999, 1))

In [458]:
X_q = X.loc[origin_q_train_user,:]
y_q = y.loc[origin_q_test_user, :]

In [483]:
X_nq = X.drop(origin_q_train_user, axis=0)
y_nq = y.drop(origin_q_test_user, axis=0)

# Survived Features from Feature Collection 1

### from Error_log

#### 보유한 model의 개수

In [163]:
train_model_count = train_e[['user_id', 'model_nm']].drop_duplicates().groupby('user_id').count()
test_model_count = test_e[['user_id', 'model_nm']].drop_duplicates().groupby('user_id').count()

#### 보유한 fwver 의 개수

In [164]:
train_fwver_count = train_e[['user_id', 'fwver']].drop_duplicates().groupby('user_id').count()
test_fwver_count = test_e[['user_id', 'fwver']].drop_duplicates().groupby('user_id').count()

#### 경험한 error 의 총 개수

In [165]:
train_err_count = train_e.groupby('user_id')['errcode'].count()
test_err_count = test_e.groupby('user_id')['errcode'].count()

#### 경험한 각 errtype의 values의 개수

In [166]:
# 33호
train_errcode_33 = train_e[train_e.errtype == 33][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
test_errcode_33 = test_e[test_e.errtype == 33][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)

train_errcode_33.columns = ['err_33_1', 'err_33_2', 'err_33_3']
test_errcode_33.columns = ['err_33_1', 'err_33_2', 'err_33_3']

In [167]:
# 23호
train_errcode_23 = train_e[train_e.errtype == 23][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
test_errcode_23 = test_e[test_e.errtype == 23][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)

train_errcode_23.columns = ['UNKNOWN', 'ACTIVE', 'connLMP', 'connESTA', 'connTO', 'connLOCAL', 'STANDBY', 'TERMINATE']
test_errcode_23.columns = ['UNKNOWN', 'ACTIVE', 'connLMP', 'connESTA', 'connTO', 'connLOCAL', 'STANDBY', 'TERMINATE']

In [168]:
# 34호
train_errcode_34 = train_e[train_e.errtype == 34][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
test_errcode_34 = test_e[test_e.errtype == 34][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)

train_errcode_34.columns = ['err_34_1', 'err_34_2', 'err_34_3', 'err_34_4', 'err_34_5', 'err_34_6']
test_errcode_34.columns = ['err_34_1', 'err_34_2', 'err_34_3', 'err_34_4', 'err_34_5', 'err_34_6']

### from Quality_log

#### 각 quality의 표준편차

In [169]:
train_qual_std = train_q.groupby(['user_id']).std()
test_qual_std = test_q.groupby(['user_id']).std()

new_columns = ['q_std_0', 'q_std_1', 'q_std_2', 'q_std_5', 'q_std_6', 'q_std_7', 'q_std_8', 'q_std_9', 'q_std_10','q_std_11', 'q_std_12']

train_qual_std.columns = new_columns
test_qual_std.columns = new_columns

#### 기록한 quality log의 개수(12개당 1번)

In [170]:
train_qual_log = train_q.groupby('user_id')['time'].count()/12
test_qual_log = test_q.groupby('user_id')['time'].count()/12

# Survived Features from Feature Collection 2

#### quality당 순수 개수

In [171]:
# 값을 만들기 위해 새로운 df를 받아옵니다.
train_q_temp = train_q.copy()
test_q_temp = test_q.copy()

# 0 값을 전부 nan 값으로 바꿔줍니다.
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] == 0, 'quality_{}'.format(i)] = np.nan
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] == 0, 'quality_{}'.format(i)] = np.nan

# 필요없는 변수들을 버려줍니다.
train_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)
test_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)

# 그룹바이 카운트 해줍니다.
train_qual_counts = train_q_temp.groupby('user_id').count()
test_qual_counts = test_q_temp.groupby('user_id').count()

# 겹치는 컬럼명을 바꿔줍니다.
train_qual_counts.columns = ['q_c_0', 'q_c_1', 'q_c_2', 'q_c_5', 'q_c_6',
                             'q_c_7', 'q_c_8', 'q_c_9', 'q_c_10', 'q_c_11', 'q_c_12']
test_qual_counts.columns = ['q_c_0', 'q_c_1', 'q_c_2', 'q_c_5', 'q_c_6',
                             'q_c_7', 'q_c_8', 'q_c_9', 'q_c_10', 'q_c_11', 'q_c_12']

#### quality당 음수, 0, 양수에 대한 count

In [172]:
train_q_temp = train_q.copy()
test_q_temp = test_q.copy()

In [173]:
train_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)
test_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)

In [174]:
# 0 값만 count를 위해서 음수와 양수를 전부 nan으로 만들겠습니다.

for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] < 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] < 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] > 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] > 0, 'quality_{}'.format(i)] = np.nan

In [175]:
train_qual_zeroCount = train_q_temp.groupby('user_id').count()
test_qual_zeroCount = test_q_temp.groupby('user_id').count()

In [176]:
new_columns = ['q_z_c_0', 'q_z_c_1', 'q_z_c_2', 'q_z_c_5', 'q_z_c_6', 'q_z_c_7', 'q_z_c_8', 'q_z_c_9', 'q_z_c_10','q_z_c_11', 'q_z_c_12']

train_qual_zeroCount.columns = new_columns
test_qual_zeroCount.columns = new_columns

In [177]:
train_q_temp = train_q.copy()
test_q_temp = test_q.copy()

In [178]:
train_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)
test_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)

In [179]:
# 음수 값만 count를 위해서 음수와 0을 전부 nan으로 만들겠습니다.

for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] >= 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] >= 0, 'quality_{}'.format(i)] = np.nan

In [180]:
train_qual_negaCount = train_q_temp.groupby('user_id').count()
test_qual_negaCount = test_q_temp.groupby('user_id').count()

In [181]:
new_columns = ['q_n_c_0', 'q_n_c_1', 'q_n_c_2', 'q_n_c_5', 'q_n_c_6', 'q_n_c_7', 'q_n_c_8', 'q_n_c_9', 'q_n_c_10','q_n_c_11', 'q_n_c_12']

train_qual_negaCount.columns = new_columns
test_qual_negaCount.columns = new_columns

# Train & Predict

### Base set

In [184]:
X = train_e.groupby(['user_id', 'errtype'])['errcode'].count().unstack().fillna(0)
y = test_e.groupby(['user_id', 'errtype'])['errcode'].count().unstack().fillna(0)

X.columns = ['et_1', 'et_2', 'et_3', 'et_4', 'et_5', 'et_6', 'et_7', 'et_8', 'et_9', 'et_10', 'et_11', 'et_12', 'et_13',
 'et_14', 'et_15', 'et_16', 'et_17', 'et_18', 'et_19', 'et_20', 'et_21', 'et_22', 'et_23', 'et_24', 'et_25',
 'et_26', 'et_27', 'et_28', 'et_30', 'et_31', 'et_32', 'et_33', 'et_34', 'et_35', 'et_36', 'et_37', 'et_38',
 'et_39', 'et_40', 'et_41', 'et_42']
y.columns = ['et_1', 'et_2', 'et_3', 'et_4', 'et_5', 'et_6', 'et_7', 'et_8', 'et_9', 'et_10', 'et_11', 'et_12', 'et_13',
 'et_14', 'et_15', 'et_16', 'et_17', 'et_18', 'et_19', 'et_20', 'et_21', 'et_22', 'et_23', 'et_24', 'et_25',
 'et_26', 'et_27', 'et_28', 'et_30', 'et_31', 'et_32', 'et_33', 'et_34', 'et_35', 'et_36', 'et_37', 'et_38',
 'et_39', 'et_40', 'et_41', 'et_42']

In [185]:
# 하나 사라진 유저를 채워주는 코드입니다.
y = y.reindex(pd.RangeIndex(y.index.max() + 1)).ffill(0)[30000:]

In [186]:
X.shape, y.shape

((15000, 41), (14999, 41))

### 1차 합병

In [187]:
X = pd.concat([X,
               train_err_count,
               train_fwver_count,
               train_model_count,
               train_qual_std,
               train_qual_log,
               train_errcode_23,
               train_errcode_33,
               train_errcode_34], axis=1).fillna(0)
X.shape

(15000, 73)

In [188]:
y = pd.concat([y,
               test_err_count,
               test_fwver_count,
               test_model_count,
               test_qual_std,
               test_qual_log,
               test_errcode_23,
               test_errcode_33,
               test_errcode_34], axis=1).fillna(0)
y.shape

(14999, 73)

### 2차 합병

In [189]:
X = pd.concat([X, train_qual_counts, train_qual_negaCount, train_qual_zeroCount], axis=1).fillna(0)
y = pd.concat([y, test_qual_counts, test_qual_negaCount, test_qual_zeroCount], axis=1).fillna(0)

X.shape, y.shape

((15000, 106), (14999, 106))

In [190]:
# X.to_csv('./edge_train.csv', index=False)
# y.to_csv('./edge_test.csv', index=False)

### 2차 면접중

In [191]:
X_ec = pd.concat([X, train_errcode_uniqueCount], axis=1).fillna(0)
y_ec = pd.concat([y, test_errcode_uniqueCount], axis=1).fillna(0)

X_ec.shape, y_ec.shape

((15000, 107), (14999, 107))

In [192]:
X_et = pd.concat([X, train_errtype_uniqueCount], axis=1).fillna(0)
y_et = pd.concat([y, test_errtype_uniqueCount], axis=1).fillna(0)

X_et.shape, y_et.shape

((15000, 107), (14999, 107))

In [193]:
X_etc = pd.concat([X, train_errcode_uniqueCount, train_errtype_uniqueCount], axis=1).fillna(0)
y_etc = pd.concat([y, test_errcode_uniqueCount, test_errtype_uniqueCount], axis=1).fillna(0)

X_etc.shape, y_etc.shape

((15000, 108), (14999, 108))

## Model

In [194]:
#------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#------------------------------------------------------------


def s_fold_train_pred(train_x, train_y):
    import lightgbm as lgb

    # Train
    models     = []
    recalls    = []
    precisions = []
    auc_scores   = []
    threshold = 0.5
    # 파라미터 설정
    params =      {
                    'boosting_type' : 'gbdt',
                    'objective'     : 'binary',
                    'metric'        : 'auc',
                    'learning_rate' : 0.027,
                    'seed': 42
                    }
    #-------------------------------------------------------------------------------------
    # 5 Kfold cross validation
    s_fold = StratifiedKFold(n_splits=5, shuffle=True ,random_state=42)    

    for train_idx, val_idx in s_fold.split(train_x, train_y):

        # split train, validation set
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx]
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx]

        d_train= lgb.Dataset(X, y)
        d_val  = lgb.Dataset(valid_x, valid_y)

        #run traning
        model = lgb.train(
                            params,
                            train_set       = d_train,
                            num_boost_round = 10000,
                            valid_sets      = d_val,
                            feval           = f_pr_auc,
                            verbose_eval    = 100, 
                            early_stopping_rounds = 100
                           )

        # cal valid prediction
        valid_prob = model.predict(valid_x)
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        # cal scores
        recall    = recall_score(    valid_y, valid_pred)
        precision = precision_score( valid_y, valid_pred)
        auc_score = roc_auc_score(   valid_y, valid_prob)

        # append scores
        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)

        print('==========================================================')
        
    return models, auc_scores, recalls, precisions

In [195]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = s_fold_train_pred(X, train_b_p.target)
print(np.mean(auc_scores))

[LightGBM] [Info] Number of positive: 4000, number of negative: 8000
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004198 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13041
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 101
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333333 -> initscore=-0.693147
[LightGBM] [Info] Start training from score -0.693147
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.826432	valid_0's pr_auc: 0.744986
[200]	valid_0's auc: 0.828085	valid_0's pr_auc: 0.75092
Early stopping, best iteration is:
[183]	valid_0's auc: 0.828701	valid_0's pr_auc: 0.752009
[LightGBM] [Info] Number of positive: 4000, number of negative: 8000
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003607 seconds.
You can set `force_col_wise=true` to remove the overhead.
[L

In [200]:
# 학습용
models, auc_scores, _, _ = s_fold_train_pred(X_et, train_b_p.target)
print(np.mean(auc_scores))

[LightGBM] [Info] Number of positive: 4000, number of negative: 8000
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004000 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13074
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 102
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333333 -> initscore=-0.693147
[LightGBM] [Info] Start training from score -0.693147
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.824661	valid_0's pr_auc: 0.741399
[200]	valid_0's auc: 0.8286	valid_0's pr_auc: 0.749177
[300]	valid_0's auc: 0.828925	valid_0's pr_auc: 0.751265
[400]	valid_0's auc: 0.828909	valid_0's pr_auc: 0.752326
Early stopping, best iteration is:
[354]	valid_0's auc: 0.829402	valid_0's pr_auc: 0.752243
[LightGBM] [Info] Number of positive: 4000, number of negative: 8000
[LightGBM] [Warning] Auto-choosing col-wise multi-thre

In [201]:
# base 0.8292073  0.8266209  0.82655950
# ec   0.8296051  0.8269322  0.82608215
# et   0.8291249  0.8269684  0.82844335
# etc  0.8291123  0.8272698  0.82821499
#          ec         etc        et

In [202]:
pred_y_list = []
for model in models:
    pred_y = model.predict(y_et)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [203]:
submission.problem = pred_ensemble
submission

user_id  problem
0        30000     0.93
1        30001     0.27
2        30002     0.55
3        30003     0.63
4        30004     0.85
...        ...      ...
14994    44994     0.24
14995    44995     0.37
14996    44996     0.76
14997    44997     0.88
14998    44998     0.34

[14999 rows x 2 columns]

In [204]:
submission.to_csv(r'C:\Users\Wyatt\wyatt37/Comp/LG_edge_detect/king/submission/king_210127_2_unique-errtype.csv', index=False)

## AutoML test

In [67]:
import lazypredict

In [68]:
from lazypredict.Supervised import LazyClassifier
X_train, X_test, y_train, y_test = train_test_split(X, train_b_p.target,
                                                    test_size = .3,
                                                    random_state = 42)
clf = LazyClassifier(verbose=0,
                     ignore_warnings=True,
                     custom_metric=None)

models, predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [01:04<00:00,  2.16s/it]


Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
LGBMClassifier                     0.80               0.73     0.73      0.79   
XGBClassifier                      0.79               0.73     0.73      0.78   
ExtraTreesClassifier               0.79               0.72     0.72      0.78   
RandomForestClassifier             0.80               0.72     0.72      0.78   
AdaBoostClassifier                 0.79               0.71     0.71      0.77   
BaggingClassifier                  0.78               0.71     0.71      0.77   
SVC                                0.79               0.70     0.70      0.77   
LinearSVC                          0.79               0.70     0.70      0.77   
NuSVC                              0.78               0.70     0.70      0.77   
LogisticRegression                 0.79               0.70     0.70      0.77   
SGDClassifier                      0.77               0.69     0.69      0.76   
BernoulliNB                        0.72               0.69     0.69      0.72   
CalibratedClassifierCV             0.78               0.68     0.68      0.75   
KNeighborsClassifier               0.76               0.68     0.68      0.74   
NearestCentroid                    0.71               0.67     0.67      0.71   
Perceptron                         0.72               0.67     0.67      0.72   
LinearDiscriminantAnalysis         0.77               0.67     0.67      0.75   
QuadraticDiscriminantAnalysis      0.76               0.67     0.67      0.74   
RidgeClassifierCV                  0.77               0.67     0.67      0.74   
RidgeClassifier                    0.77               0.67     0.67      0.74   
DecisionTreeClassifier             0.70               0.65     0.65      0.70   
ExtraTreeClassifier                0.68               0.64     0.64      0.68   
LabelSpreading                     0.71               0.64     0.64      0.70   
LabelPropagation                   0.71               0.64     0.64      0.70   
PassiveAggressiveClassifier        0.66               0.64     0.64      0.67   
GaussianNB                         0.74               0.62     0.62      0.70   
CheckingClassifier                 0.68               0.50     0.50      0.55   
DummyClassifier                    0.56               0.50     0.50      0.56   

                               Time Taken  
Model                                      
LGBMClassifier                       0.31  
XGBClassifier                        0.81  
ExtraTreesClassifier                 1.87  
RandomForestClassifier               2.39  
AdaBoostClassifier                   1.37  
BaggingClassifier                    2.61  
SVC                                  7.91  
LinearSVC                            3.83  
NuSVC                               10.07  
LogisticRegression                   0.16  
SGDClassifier                        0.15  
BernoulliNB                          0.09  
CalibratedClassifierCV              13.74  
KNeighborsClassifier                 4.18  
NearestCentroid                      0.06  
Perceptron                           0.07  
LinearDiscriminantAnalysis           0.21  
QuadraticDiscriminantAnalysis        0.18  
RidgeClassifierCV                    0.16  
RidgeClassifier                      0.07  
DecisionTreeClassifier               0.42  
ExtraTreeClassifier                  0.08  
LabelSpreading                       7.24  
LabelPropagation                     6.48  
PassiveAggressiveClassifier          0.08  
GaussianNB                           0.08  
CheckingClassifier                   0.06  
DummyClassifier                      0.06

# Evaluation

### 1,2차 합본

0.8075 -> 0.8161로 상승했습니다. 그래도 0.9% 오른거면 꽤나 올랐네요!

### 1,2차 합본 파라미터 튜닝

파라미터 튜닝을 해줬는데, 0.8161 -> 0.8222	로 상승했습니다. lgbm 만세! 0.6% 올랐습니다.

바꾼 파라미터

1. lr {'default' : 0.027}
2. num_boost_round {1000:10000}
3. early_stopping_rounds {3 :100}

미세하게 많이 개선하도록 바꿨습니다.

### 3차 합본

specific error count 를 넣으니 성능이 오히려 떨어졌습니다.

0.8217로 큰 차이는 없으나 Loss를 따라가는 것 같습니다.

하나씩 줄여가면서 다시 학습을 시켜보겠습니다.

하나씩 학습시켜보았으나, 기존의 로스를 넘는 변수는 없었음

### 4차 합본

0.8222 -> 0.8228 로 아주 미세하게 상승 0.0006 상승했습니다.

### 5차 합본

1. 'conn' 이 포함된 errcode의 갯수: 기각
2. robot 형식의 errcode의 갯수: 기각
3. 'android'가 들어간 errcode의 갯수: 기각

안드로이드는 0.8228 -> 0.8230 으로 아주 미세하게 0.0002 상승했습니다. 그러나 기각시켰습니다. 이유는 fc1에 있습니다.

### 오버샘플링

1. Over Sampling by SMOTE-B: 이상한 로스가 나오면서 리더보드 떨어짐

### 6차 합본

1. 23호, 33호, 34호가 살아남았습니다.

### 7차 합본

1. quality count: 각 퀄리티별 0이 아닌 로그 개수 ->>> 로스는 떨어졌으나, 리더보드 아주 미세하게 오름.
2. quality time gap: 유저별 퀄리티 로그의 발생 간격의 합 ->>> 로스도, 리더보드도 아주 미세하게 떨어짐.
3. quality time continue: 유저별 퀄리티 로그의 연속 발생 횟수(2시간 뒤에 바로 2시간이 기록되면 1을 더해줌) ->>> 로스도, 리더보드도 아주 미세하게 떨어짐
4. quality onehot: 0을 제외한 값을 전부 1로 바꿔서 groupby().max() 값으로 뽑았음 ->>> 로스가 너무 떨어져서 제출 기각
5. quality mean: 음...std랑 함께 넣으면 더 나아질까 싶어 넣어봤습니다.->>> 로스는 올랐으나, 리더보드는 미세하게 떨어짐.

### 변수 제거

1. 8호와 9호를 제거해보았습니다. 리더보드 떨어졌습니다.

### Q유저와 NQ유저 별도 모델 학습

1. 기각

### 8차 합본

1. qual_zero_count(40): 0.83199
2. qual_posi_count(40): 0.83140
3. qual_nega_count(40): 0.83204
4. qual_zp_count(42):   0.83156
5. qual_pn_count(42):   0.83274
6. qual_nz_count(42):   0.83277
7. qual_zpn_count(42):  0.83241

In [1]:
#               42          41          40
# zero ->>> 0.82865770, 0.83041505, 0.82704965            -> 2차-2등  -> 3차-3등
# posi ->>> 0.82861785, 0.82928790, 0.82661170
# nega ->>> 0.82874720, 0.82886705, 0.82726769 -> 1차-3등             -> 3차-2등
# zp ->>>>> 0.82711640, 0.83009640, 0.82695850            -> 2차-1등
# pn ->>>>> 0.82884685, 0.82890579. 0.82632940 -> 1차-2등
# nz ->>>>> 0.82920730, 0.82957185, 0.82653675 -> 1차-1등
# zpn ->>>> 0.82869589, 0.82983390, 0.82762975            -> 2차-3등  -> 3차-1등

random_state 값을 다르게 주어 제출했습니다. 그것도 비교를 해보아야겠습니다.<br>
42 시드가 리더보드와 가장 유사합니다. 다만 프라이빗에 오버피팅될 수도 있으니 유의해야겠습니다.

### 9차 합본